### Parametric Value at Risk (VaR) Using the Yield Series

Parametric VaR is a statistical measure employed to estimate the maximum potential loss in the value of a portfolio over a specified time horizon (e.g., one day) at a given confidence level (e.g., 95%), under the assumption that asset returns follow a normal distribution. This method is particularly useful in portfolio analysis where it supports scenario analysis, risk reporting, and strategic decision-making by quantifying downside risks in yield-sensitive instruments.

The dataset "BAMLEM2BRRBBBCRPIEY.csv" contains daily yield observations spanning from November 23, 2020, to November 21, 2025 (1,292 valid daily changes after cleaning). Yields represent the annualized return on fixed-income securities; adverse movements (e.g., rising yields) inversely impact bond prices. For this analysis, parametric VaR is computed on daily yield changes (differences between consecutive yields), as these capture short-term volatility relevant to liquidity and capital utilization assessments.

#### Parametric VaR Formula

The parametric VaR at confidence level $1 - \alpha$ (e.g., 95%, so $\alpha = 0.05$) for a one-day horizon is:
$$\text{VaR}_\alpha = \mu - \sigma \cdot z_\alpha$$
where:

- $\mu$: The mean of daily yield changes (arithmetic average, representing expected daily movement).
- $\sigma$: The standard deviation of daily yield changes (a measure of volatility or dispersion).
- $z_\alpha$: The z-score from the standard normal distribution (e.g., 1.645 for 95% confidence, indicating the threshold where 5% of outcomes exceed the value in the left tail).
- The result is a negative value, interpretable as the potential adverse yield increase (in decimal form; multiply by 100 for percentage) at the confidence level. For a bond portfolio, this can be translated to price VaR using duration: Approximate Price VaR $\approx - \text{Duration} \times \text{Yield VaR}$, where Duration is the sensitivity measure (not computed here, assuming a unit position for simplicity).


#### Computed Results
Based on the dataset:

- Number of daily yield changes: 1,292.
- $\mu = 0.001687$ (0.1687% expected daily increase).
- $\sigma = 0.047269$ (4.7269% daily volatility).


| Confidence Level | $z_\alpha$ | Parametric VaR (Daily Yield Change) |
|------------------|-------------|-------------------------------------|
| 95%             | 1.645      | -0.07607 (-7.61%)                   |
| 99%             | 2.326      | -0.1083 (-10.83%)                  |

**Interpretation**: At 95% confidence, there is a 5% chance that daily yield changes could exceed a 7.61% increase (worsening for long bond positions). The higher 99% threshold (10.83%) reflects tail risks, useful for stress testing in emerging market bond funds. These values assume normality.


In [1]:
import pandas as pd  
import numpy as np   


df = pd.read_csv('/Users/bekay/Documents/Studies/MSc Financial Engineering/Models/BAMLEM2BRRBBBCRPIEY.csv')  


In [2]:
df.head()

,Date,BAMLEM2BRRBBBCRPIEY
0,23/11/2020,2.68
1,24/11/2020,2.68
2,25/11/2020,2.68
3,26/11/2020,2.67
4,27/11/2020,2.65


In [3]:
# Parse the Date column as datetime, handling any parsing issues.
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')

In [4]:
# Replace empty strings or blanks in the yield column with NaN for numeric conversion.
df['BAMLEM2BRRBBBCRPIEY'] = pd.to_numeric(df['BAMLEM2BRRBBBCRPIEY'], errors='coerce')

In [5]:
#Sort by date to ensure chronological order.
df = df.sort_values('Date').reset_index(drop=True)

In [6]:
# Compute daily yield changes (differences).
df['Yield_Change'] = df['BAMLEM2BRRBBBCRPIEY'].diff()

In [7]:
# Drop rows with NaN in Yield_Change.
df_clean = df.dropna(subset=['Yield_Change'])

In [8]:
# Compute mean and standard deviation of yield changes.
mu = df_clean['Yield_Change'].mean()  
sigma = df_clean['Yield_Change'].std()  

In [9]:
# Z-scores for confidence levels.
z_alpha = 1.645  
z_99 = 2.326

# Compute VaR values.
var_95 = mu - sigma * z_alpha  
var_99 = mu - sigma * z_99

# Output key statistics.
print(f"Number of observations: {len(df_clean)}") 
print(f"Mean daily yield change (μ): {mu:.6f}")   
print(f"Standard deviation (σ): {sigma:.6f}")
print(f"95% Parametric VaR: {var_95:.6f}")
print(f"99% Parametric VaR: {var_99:.6f}")

Number of observations: 1292
Mean daily yield change (μ): 0.001687
Standard deviation (σ): 0.047269
95% Parametric VaR: -0.076070
99% Parametric VaR: -0.108260


In [10]:
# Summary statistics.
print(df_clean['Yield_Change'].describe())

count    1292.000000
mean        0.001687
std         0.047269
min        -0.870000
25%        -0.020000
50%         0.000000
75%         0.020000
max         0.350000
Name: Yield_Change, dtype: float64


### Parametric Value at Risk for Bond Prices Incorporating Duration

In fixed-income portfolio management, duration serves as a critical measure of interest rate sensitivity. Macaulay duration, expressed in years, approximates the weighted average time to receive cash flows from a bond, thereby quantifying the percentage change in bond price for a 1% (100 basis point) change in yield. Bond prices and yields move inversely: an increase in yield leads to a decrease in price, with the magnitude governed by duration. The modified duration formula for price sensitivity is:

$$\Delta P / P \approx -D \times \Delta y$$
where:

$\Delta P / P$: The relative change in bond price (decimal).
$D$: Modified duration (in years; often approximated by Macaulay duration for small yield changes).
$\Delta y$: Change in yield (decimal, e.g., 0.01 for 1%).

To incorporate duration into parametric VaR, the yield VaR (from the previous analysis) is scaled by duration to derive the price VaR. The adverse yield change (positive $\Delta y$) corresponds to a price loss, yielding:
$$\text{Price VaR}_\alpha \approx D \times (\sigma \cdot z_\alpha - \mu)$$
where $\mu$ is the mean daily yield change, $\sigma$ is the standard deviation of daily yield changes, and $z_\alpha$ is the z-score (e.g., 1.645 for 95% confidence). This assumes a long bond position; the result is a positive value representing potential percentage loss at confidence level $1 - \alpha$.


The dataset BAMLEM2BRRBBBCRPIEY.csv tracks the ICE BofA BBB Emerging Markets Corporate Plus Index Effective Yield, a benchmark for BBB-rated USD-denominated emerging market corporate bonds. Based on market data, the average duration for broad emerging market corporate bonds is approximately 5.3 years. This value is used here for illustration.

#### Computed Results

Using the dataset's statistics ($\mu = 0.001687$, $\sigma = 0.047269$, 1,292 observations):

Adverse daily yield change at 95% confidence: $0.047269 \times 1.645 - 0.001687 = 0.0761$ (7.61% increase).
Price VaR at 95% confidence (with $D = 5.3$): $5.3 \times 0.0761 = 0.403$ (40.3% potential price decline).

| Confidence Level | $z_\alpha$ | Yield VaR (Daily Change) | Price VaR (Daily % Loss, $D=5.3$) |
|------------------|-------------|---------------------------|------------------------------------|
| 95%             | 1.645      | -0.0761 (-7.61%)         | 0.4032 (40.3%)                     |
| 99%             | 2.326      | -0.1083 (-10.83%)        | 0.5738 (57.4%)                     |

**Interpretation**: At 95% confidence, there is a 5% probability of a daily bond price decline exceeding 40.3% due to yield increases, assuming a 5.3-year duration. This elevated figure reflects the high volatility ($\sigma = 4.73\%$) in emerging market yields over the 2020-2025 period, influenced by events like inflation surges and geopolitical tensions. Such VaR informs liquidity buffers and scenario analyses, with mitigations like diversification or hedging against natural disasters or political risks.risks.



In [11]:
# Z-scores.
z_95 = 1.645  # 95% confidence (left-tail threshold).
z_99 = 2.326  # 99% confidence.

In [12]:
# Yield VaR (negative for adverse move).
yield_var_95 = mu - sigma * z_95  
yield_var_99 = mu - sigma * z_99

In [13]:
# Assume duration D = 5.3 years (from market data).
D = 5.3  # Scalar float.

# Price VaR: D * adverse yield change = D * (-yield_var).
price_var_95 = D * (-yield_var_95)  # Positive loss percentage.
price_var_99 = D * (-yield_var_99)

# Output.
print(f"Mean daily yield change (μ): {mu:.6f}")
print(f"Standard deviation (σ): {sigma:.6f}")
print(f"95% Yield VaR: {yield_var_95:.4f} ({yield_var_95*100:.2f}%)")
print(f"95% Price VaR (D={D}): {price_var_95:.4f} ({price_var_95*100:.2f}%)")
print(f"99% Yield VaR: {yield_var_99:.4f} ({yield_var_99*100:.2f}%)")
print(f"99% Price VaR (D={D}): {price_var_99:.4f} ({price_var_99*100:.2f}%)")

Mean daily yield change (μ): 0.001687
Standard deviation (σ): 0.047269
95% Yield VaR: -0.0761 (-7.61%)
95% Price VaR (D=5.3): 0.4032 (40.32%)
99% Yield VaR: -0.1083 (-10.83%)
99% Price VaR (D=5.3): 0.5738 (57.38%)
